In [1]:
# Kiểm tra PyTorch đã nhận GPU chưa
import torch
print('CUDA available:', torch.cuda.is_available())
print('CUDA device count:', torch.cuda.device_count())

# Kiểm tra current device chỉ khi CUDA available
if torch.cuda.is_available():
    print('Current device:', torch.cuda.current_device())
    print('Device name:', torch.cuda.get_device_name(torch.cuda.current_device()))
else:
    print('CUDA không khả dụng, không thể lấy thông tin device')

# Kiểm tra Ultralytics YOLO
import ultralytics
print('Ultralytics version:', ultralytics.__version__)


CUDA available: False
CUDA device count: 0
CUDA không khả dụng, không thể lấy thông tin device


ModuleNotFoundError: No module named 'ultralytics'

In [ ]:
## Kiểm tra đường dẫn Python và PyTorch
import sys
import os

print("Python path:", sys.executable)
print("Python version:", sys.version)

try:
    import torch
    print("PyTorch path:", torch.__file__)
    print("PyTorch version:", torch.__version__)
    print("CUDA available:", torch.cuda.is_available())

    if torch.cuda.is_available():
        print("CUDA version:", torch.version.cuda)
    else:
        print("CUDA không được hỗ trợ bởi PyTorch hiện tại")
except ImportError:
    print("PyTorch chưa được cài đặt")

In [ ]:
BASE_PATH = "C:\\Users\\Seotow\\OneDrive - Thang Long University\\Hieu\\Learn\\ThirdYear\\MachineLearning\\BTL"
from ultralytics import YOLO

# Đường dẫn file cấu hình dữ liệu
DATASET_YAML = os.path.join(BASE_PATH, 'vietnamese-license-plates', 'dataset.yaml')

# Khởi tạo model YOLOv8n (có thể thay bằng yolov8s, yolov8m...)
model = YOLO('yolov8n.pt')

print("Dataset configuration:")
with open(DATASET_YAML, 'r') as f:
    print(f.read())


In [ ]:
from ultralytics import YOLO

# Đường dẫn file cấu hình dữ liệu
DATASET_YAML = '../vietnamese-license-plates/dataset.yaml'

# Khởi tạo model YOLOv8n (có thể thay bằng yolov8s, yolov8m...)
model = YOLO('yolov8n.pt')

# Train model
results = model.train(
    data=DATASET_YAML,
    epochs=50,           # Số epoch, có thể tăng/giảm
    imgsz=640,           # Kích thước ảnh
    batch=16,            # Batch size, điều chỉnh theo VRAM
    device=0,            # 0: GPU đầu tiên
    project='runs/detect', # Thư mục lưu kết quả
    name='yolo-plates-custom', # Tên folder/model
    exist_ok=True        # Ghi đè nếu đã tồn tại
)

# Lưu model đã train
model.save('yolo-plates-custom.pt')
print('Đã train xong và lưu model tại runs/detect/yolo-plates-custom/')

In [ ]:
# 1. Tải mô hình đã train
from ultralytics import YOLO
import os

# Đường dẫn đến model đã train
# Thay đổi model_path tùy theo bạn chọn model GPU hay CPU
# Model GPU
model_path = 'runs/detect/yolo-plates-custom/weights/best.pt'
# Model CPU (nếu bạn đã train với CPU)
# model_path = 'runs/detect/yolo-plates-cpu/weights/best.pt'

# Kiểm tra xem model có tồn tại không
if os.path.exists(model_path):
    print(f"Đã tìm thấy model tại: {model_path}")
    # Tải model
    model = YOLO(model_path)
else:
    print(f"Không tìm thấy model tại: {model_path}")
    print("Đang sử dụng model yolo-plates-custom.pt nếu có")
    if os.path.exists('yolo-plates-custom.pt'):
        model = YOLO('yolo-plates-custom.pt')
    else:
        print("Không tìm thấy model đã train, sử dụng model mặc định yolov8n.pt")
        model = YOLO('yolov8n.pt')

print("Đã tải xong model")

In [ ]:
# 2. Chuẩn bị ảnh test
import cv2
import matplotlib.pyplot as plt
import os
import glob
from PIL import Image
import numpy as np

# Danh sách các tệp ảnh thử nghiệm có sẵn trong thư mục gốc
test_images = ['1.jpg', '11.jpg', '3.jpg', '745.jpg', '9145.jpg']

# Kiểm tra và tạo danh sách các ảnh có sẵn
available_images = []
for img_file in test_images:
    full_path = os.path.join('..', img_file)
    if os.path.exists(full_path):
        available_images.append(full_path)
        print(f"Tìm thấy ảnh test: {full_path}")
    else:
        print(f"Không tìm thấy ảnh: {full_path}")

# Nếu không tìm thấy ảnh nào, tìm kiếm trong thư mục val
if not available_images:
    print("Không tìm thấy ảnh test trong thư mục gốc, tìm kiếm trong thư mục val...")
    val_images = glob.glob('../vietnamese-license-plates/images/val/*.jpg')[:5]
    if val_images:
        available_images = val_images
        for img in available_images:
            print(f"Sử dụng ảnh val: {img}")
    else:
        print("Không tìm thấy ảnh nào trong thư mục val")

# Hiển thị ảnh đầu tiên nếu có
if available_images:
    img = cv2.imread(available_images[0])
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10, 8))
    plt.imshow(img_rgb)
    plt.title("Ảnh test đầu tiên")
    plt.axis('off')
    plt.show()
else:
    print("Không tìm thấy ảnh test nào để hiển thị")

In [ ]:
# 3. Thực hiện dự đoán
import torch

if not available_images:
    print("Không có ảnh test nào để dự đoán")
else:
    # Chọn thiết bị để dự đoán (GPU hoặc CPU)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Sử dụng thiết bị: {device}")

    # Thực hiện dự đoán trên ảnh đầu tiên
    first_image = available_images[0]
    print(f"Đang dự đoán trên ảnh: {first_image}")

    # Thực hiện dự đoán
    results = model.predict(first_image, conf=0.25, device=device)  # conf là ngưỡng tin cậy

    print(f"Đã dự đoán xong, tìm thấy {len(results[0].boxes)} đối tượng")

    # In thông tin chi tiết về các đối tượng phát hiện được
    for i, box in enumerate(results[0].boxes):
        confidence = box.conf.item()
        cls_id = int(box.cls.item())
        cls_name = results[0].names[cls_id]
        print(f"Đối tượng {i+1}: {cls_name}, độ tin cậy: {confidence:.2f}")

In [ ]:
# 4. Hiển thị kết quả
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from PIL import Image, ImageDraw, ImageFont

if not available_images or len(results[0].boxes) == 0:
    print("Không có kết quả để hiển thị")
else:
    # Hiển thị kết quả theo cách 1: Dùng plot từ model
    plt.figure(figsize=(12, 10))
    img_result = results[0].plot()
    plt.imshow(cv2.cvtColor(img_result, cv2.COLOR_BGR2RGB))
    plt.title("Kết quả nhận diện")
    plt.axis('off')
    plt.show()

    # Hiển thị kết quả theo cách 2: Tự vẽ các bounding box
    img = cv2.imread(first_image)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(12, 10))
    plt.imshow(img_rgb)

    # Vẽ các bounding box
    ax = plt.gca()

    for box in results[0].boxes:
        # Lấy tọa độ
        x1, y1, x2, y2 = box.xyxy[0].tolist()

        # Lấy thông tin class và độ tin cậy
        cls_id = int(box.cls.item())
        cls_name = results[0].names[cls_id]
        confidence = box.conf.item()

        # Tạo rectangle
        rect = patches.Rectangle(
            (x1, y1), x2-x1, y2-y1,
            linewidth=2, edgecolor='r', facecolor='none'
        )
        ax.add_patch(rect)

        # Thêm nhãn
        plt.text(
            x1, y1-10,
            f"{cls_name}: {confidence:.2f}",
            color='red', fontsize=12, backgroundcolor='white'
        )

    plt.title("Kết quả nhận diện (tự vẽ)")
    plt.axis('off')
    plt.show()

In [ ]:
# 5. Thử nghiệm trên nhiều ảnh
def predict_multiple_images(image_list, model, conf=0.25, max_images=5):
    """Dự đoán và hiển thị kết quả trên nhiều ảnh"""
    if not image_list:
        print("Không có ảnh để dự đoán")
        return

    # Giới hạn số lượng ảnh để hiển thị
    image_list = image_list[:max_images]

    # Thiết bị
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # Dự đoán trên tất cả các ảnh
    for img_path in image_list:
        print(f"\nĐang dự đoán trên ảnh: {img_path}")

        # Thực hiện dự đoán
        result = model.predict(img_path, conf=conf, device=device)[0]

        print(f"Tìm thấy {len(result.boxes)} đối tượng")

        # Hiển thị kết quả
        plt.figure(figsize=(12, 10))
        img_result = result.plot()
        plt.imshow(cv2.cvtColor(img_result, cv2.COLOR_BGR2RGB))
        plt.title(f"Kết quả: {os.path.basename(img_path)}")
        plt.axis('off')
        plt.show()

        # In thông tin về các đối tượng
        for i, box in enumerate(result.boxes):
            confidence = box.conf.item()
            cls_id = int(box.cls.item())
            cls_name = result.names[cls_id]
            print(f"Đối tượng {i+1}: {cls_name}, độ tin cậy: {confidence:.2f}")

# Kiểm tra xem có ảnh test để thử nghiệm không
if available_images and len(available_images) > 1:
    print(f"Thử nghiệm trên {len(available_images)} ảnh...")
    predict_multiple_images(available_images, model, conf=0.25)
else:
    print("Không đủ ảnh để thử nghiệm thêm")

In [ ]:
# 6. Đánh giá hiệu suất mô hình
# Kiểm tra hiệu suất của mô hình trên tập validation

# Đường dẫn đến tập validation
val_dir = '../vietnamese-license-plates/images/val'

# Kiểm tra nếu tập validation tồn tại
if os.path.exists(val_dir):
    print(f"Đánh giá mô hình trên tập validation: {val_dir}")

    # Thực hiện đánh giá
    results = model.val(data=DATASET_YAML, split='val')

    # In kết quả đánh giá
    metrics = results.box
    print("\nKết quả đánh giá mô hình:")
    print(f"mAP@50: {metrics.map50}")
    print(f"mAP@50-95: {metrics.map}")
    print(f"Precision: {metrics.p}")
    print(f"Recall: {metrics.r}")
    print(f"F1-Score: {metrics.f1}")
else:
    print(f"Không tìm thấy thư mục validation: {val_dir}")
    print("Bỏ qua bước đánh giá hiệu suất mô hình")